In [1]:
!git clone https://github.com/trduc97/neural_medical_qa.git
    
%cd neural_medical_qa
!pip install -r requirements.txt

Cloning into 'neural_medical_qa'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 76 (delta 29), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (76/76), 1.75 MiB | 3.84 MiB/s, done.
/kaggle/working/neural_medical_qa


In [2]:
!pip install transformers

# 0. Dataset

In [3]:
from import_datasets import load_bioasq_pubmedqa,  train_val_test_split

bioasq, pubmedqa = load_bioasq_pubmedqa()

# Display the first few samples of the PubMedQA dataset
print(pubmedqa['train'].to_pandas().head())

responses = pubmedqa['train']['final_decision']
# Counting the occurrences of each value
yes_count = responses.count('yes')
no_count = responses.count('no')
maybe_count = responses.count('maybe')

# Display the counts
print(f"Yes: {yes_count}")
print(f"No: {no_count}")
print(f"Maybe: {maybe_count}")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

      pubid                                           question  \
0  21645374  Do mitochondria play a role in remodelling lac...   
1  16418930  Landolt C and snellen e acuity: differences in...   
2   9488747  Syncope during bathing in infants, a pediatric...   
3  17208539  Are the long-term results of the transanal pul...   
4  10808977  Can tailored interventions increase mammograph...   

                                             context  \
0  {'contexts': ['Programmed cell death (PCD) is ...   
1  {'contexts': ['Assessment of visual acuity dep...   
2  {'contexts': ['Apparent life-threatening event...   
3  {'contexts': ['The transanal endorectal pull-t...   
4  {'contexts': ['Telephone counseling and tailor...   

                                         long_answer final_decision  \
0  Results depicted mitochondrial dynamics in viv...            yes   
1  Using the charts described, there was only a s...             no   
2  "Aquagenic maladies" could be a pediatric form... 

# 1. Split the dataset

In [4]:
pubmedqa_train,pubmedqa_val, pubmedqa_test = train_val_test_split(pubmedqa)
print(f"Train size: {len(pubmedqa_train)}")
print(f"Validation size: {len(pubmedqa_val)}")
print(f"Test size: {len(pubmedqa_test)}")

Train size: 750
Validation size: 100
Test size: 150


# 2. EDA

In [5]:
from collections import defaultdict
# Initialize a defaultdict to hold the bucket counts
length_buckets = defaultdict(int)

# Define the bucket size
bucket_size = 128

# Loop through each string in the list
for s in pubmedqa_train['long_answer']:
    # Determine the bucket for the current string length
    bucket = (len(s) // bucket_size) * bucket_size
    # Increment the count for the appropriate bucket
    length_buckets[bucket] += 1

# Display the counts for each bucket
for bucket, count in sorted(length_buckets.items()):
    print(f"Length {bucket} - {bucket + bucket_size - 1}: {count} strings")

Length 0 - 127: 66 strings
Length 128 - 255: 326 strings
Length 256 - 383: 244 strings
Length 384 - 511: 82 strings
Length 512 - 639: 25 strings
Length 640 - 767: 4 strings
Length 768 - 895: 3 strings


# 3. Model

## 3.0. Importing libraries

In [6]:
# @title
import pickle
from sklearn.metrics import roc_auc_score
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from tensorflow.keras.layers import GlobalMaxPool2D, GlobalAvgPool2D, Multiply, Subtract, Add, Activation, MaxPool2D, Concatenate,Reshape, Conv2D, MaxPooling2D, Dropout, Input, Embedding, Bidirectional, LSTM, Dense, Lambda, GlobalAveragePooling1D, Reshape, GRU, Flatten, Dropout, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from tensorflow.keras.activations import gelu
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import zipfile
import os
import tensorflow as tf
# Setting epoch_count = 1 just to let the file run as demo, we have saved the trained models in a separated folder and will be using this to review
batch_size=256
epoch_count=10
activation_func=gelu

2024-07-26 19:46:10.174335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 19:46:10.174441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 19:46:10.308868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# List all available devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

results={}

Num GPUs Available:  1


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import class_weight
import os
import zipfile

class ModelTraining:
    def __init__(self, responses):
        # Prepare a dictionary to record result
        self.dl_results={}
        # Rebalanced classes weights
        self.class_weights = self.compute_class_weights(responses)
        # Define metrics
        self.metrics = self.define_metrics()
        self._optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5)
        self._loss_func=tf.keras.losses.CategoricalCrossentropy()
        self.histories={}

    # metrics
    def f1_score_metrics(self, y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
        predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
        recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        return f1
    
    def define_metrics(self):
        return [
            tf.keras.metrics.Accuracy(name='accuracy'),
#            tf.keras.metrics.Precision(name='precision'),
#            tf.keras.metrics.Recall(name='recall'),
#            tf.keras.metrics.AUC(name='auc'),
#            tf.keras.metrics.F1Score
#            self.f1_score_metrics,
        ]
    #Calculate class weights
    def compute_class_weights(self, responses):
        class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                          classes=np.unique(responses),
                                                          y=responses)
        # Convert class weights to a dictionary
        return dict(enumerate(class_weights))

    # Setting up function to measure the models performance
    def dl_metrics(self, test_dataset, model, model_name):
        test_results = model.evaluate(test_dataset)
        self.dl_results[model_name] = test_results

    # Code to zip
    def zip_folder(self, folder_path, zip_path):
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through all the files in the folder
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    # Add each file to the zip file
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, folder_path))
    def compile_model(self,model, model_name):
        model.compile(optimizer=self._optimizer,
                      loss=self._loss_func,
                      metrics=self.metrics)
#        model.summary()
#        model_plot = model_name +'.png'
#        plot_model(model,show_shapes=True,to_file = model_plot)
    # Code to train model
    def train_model(self, model,model_name,
                    train_dataset,
                    validate_dataset,
                    test_dataset,
                    epochs=epoch_count, batch_size=batch_size):

        history = model.fit(train_dataset,
                            epochs=epochs,
                            #batch_size=batch_size,
                            validation_data=validate_dataset,
#                            class_weight=self.class_weights,
                            )
        self.histories[model_name] = history

        # measure performance on the test dataset and save to dicitonary

        self.dl_metrics(test_dataset, model, model_name)
        # Saving the trained model
        model.save(model_name)

        # Specify the folder to be zipped and the path to save the zip file
        zip_file_path = model_name + '.zip'

        # Call the function to zip the folder
        self.zip_folder(model_name, zip_file_path)

## 3.1. Text representation

### 3.1.1. TF-IDF

### 3.1.2. Embedding layer

### 3.1.3. ELMo 

### 3.1.4. BERT (base-uncased)

In [9]:
embedding='bert'
neural_model='dense'
model_name=embedding+'_'+neural_model

In [25]:
pubmedqa_train

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision', 'decision_encoded', '__index_level_0__'],
    num_rows: 750
})

In [10]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from datasets import load_dataset

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_function(examples):
    return tokenizer(examples['question'], examples['long_answer'], truncation=True, padding='max_length', max_length=512)

# Tokenize the datasets
train_dataset = pubmedqa_train.map(preprocess_function, batched=True)
val_dataset = pubmedqa_val.map(preprocess_function, batched=True)
test_dataset = pubmedqa_test.map(preprocess_function, batched=True)

# Convert the datasets to TensorFlow datasets
def convert_to_tf_dataset(dataset, shuffle=True, batch_size=32):
    def gen():
        for ex in dataset:
            yield ({'input_ids': ex['input_ids'], 'attention_mask': ex['attention_mask']}, ex['decision_encoded'])
    
    return tf.data.Dataset.from_generator(gen, 
                                          ({'input_ids': tf.int32, 'attention_mask': tf.int32}, tf.int64),
                                          ({'input_ids': tf.TensorShape([None]), 'attention_mask': tf.TensorShape([None])}, tf.TensorShape([]))
                                         ).shuffle(1000).batch(batch_size)

train_tf_dataset = convert_to_tf_dataset(train_dataset)
val_tf_dataset = convert_to_tf_dataset(val_dataset, shuffle=False)
test_tf_dataset = convert_to_tf_dataset(test_dataset, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [21]:
# Instantiate a model training session
model_training = ModelTraining(responses)

In [22]:
class PubMedQAModel(tf.keras.Model):
    def __init__(self, model_name='bert-base-uncased',classes=3, dropout=0.3,nodes=32):
        super(PubMedQAModel, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name)      
        # Freeze all layers except the last two
        for layer in self.bert.layers:
            layer.trainable = False
        for layer in self.bert.bert.encoder.layer[-2:]:
            layer.trainable = True
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)
        self.dense1 = tf.keras.layers.Dense(nodes, activation='relu')
        self.classifier = tf.keras.layers.Dense(classes, activation='softmax')
    
    def call(self, inputs):
        input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropout_output1 = self.dropout1(pooled_output, training=False)
        dense_output1= self.dense1(dropout_output1)
        dropout_output2 = self.dropout2(dense_output1, training=False)        
        logits = self.classifier(dropout_output2)
        return logits

model = PubMedQAModel()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [23]:
model_training.compile_model(model, model_name)

In [24]:
model_training.train_model(model, model_name,
                            train_tf_dataset, 
#                            val_tf_dataset,
                            test_tf_dataset
                           )

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 3)

In [43]:
for data, label in train_tf_dataset.take(1):
    print(data)
    print(label)


{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[  101, 26351, 17635, ...,     0,     0,     0],
       [  101,  2003,  1996, ...,     0,     0,     0],
       [  101,  2041,  1997, ...,     0,     0,     0],
       ...,
       [  101,  2003,  4367, ...,     0,     0,     0],
       [  101,  2566, 12690, ...,     0,     0,     0],
       [  101,  2515,  1996, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}
tf.Tensor([2 2 2 0 0 0 2 0 0 2 2 2 2 0 2 2 0 1 2 1 1 2 0 2 2 2 2 0 0 2 0 2], shape=(32,), dtype=int64)


In [47]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_tf_dataset, validation_data=val_tf_dataset, epochs=15)

Epoch 1/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.4777 - loss: 1.0629 - val_accuracy: 0.5200 - val_loss: 1.0243
Epoch 2/15


W0000 00:00:1721948363.754266     144 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 519ms/step - accuracy: 0.5637 - loss: 0.9935 - val_accuracy: 0.5400 - val_loss: 0.9955
Epoch 3/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 519ms/step - accuracy: 0.5492 - loss: 0.9814 - val_accuracy: 0.5400 - val_loss: 0.9806
Epoch 4/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 519ms/step - accuracy: 0.5608 - loss: 0.9580 - val_accuracy: 0.5400 - val_loss: 0.9726
Epoch 5/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 519ms/step - accuracy: 0.5552 - loss: 0.9546 - val_accuracy: 0.5400 - val_loss: 0.9676
Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 520ms/step - accuracy: 0.5488 - loss: 0.9454 - val_accuracy: 0.5400 - val_loss: 0.9637
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 519ms/step - accuracy: 0.5602 - loss: 0.9408 - val_accuracy: 0.5400 - val_loss: 0.9604
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 519ms/step - accuracy: 0.5380 - loss: 0.9550 - val_accuracy: 0.5400 - val_loss: 0.9581
Epoch 9/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 519ms/step - accuracy: 0.5542 - loss: 0.9365 - val_accuracy: 0.540

In [48]:
model.evaluate(test_tf_dataset)

      4/Unknown 2s 460ms/step - accuracy: 0.5358 - loss: 0.9711

W0000 00:00:1721948577.265502     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 996ms/step - accuracy: 0.5439 - loss: 0.9633


[0.9476428031921387, 0.5600000023841858]

In [54]:
import tensorflow as tf
from transformers import TFBertModel

class PubMedQAModel(tf.keras.Model):
    def __init__(self, model_name='bert-base-uncased',classes=3, dropout=0.3,nodes=32):
        super(PubMedQAModel, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name)      
        # Freeze all layers except the last two
        for layer in self.bert.layers:
            layer.trainable = False
#        for layer in self.bert.bert.encoder.layer[-2:]:
#            layer.trainable = True
        self.dropout1 = tf.keras.layers.Dropout(dropout)
#        self.dropout2 = tf.keras.layers.Dropout(dropout)
#        self.dense1 = tf.keras.layers.Dense(nodes, activation='relu')
        self.classifier = tf.keras.layers.Dense(classes, activation='softmax')
    
    def call(self, inputs):
        input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropout_output1 = self.dropout1(pooled_output, training=False)
#        dense_output1= self.dense1(dropout_output1)
#        dropout_output2 = self.dropout2(dense_output1, training=False)        
        logits = self.classifier(dropout_output1)
        return logits

model = PubMedQAModel()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [55]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
epochs = 10

history = model.fit(train_tf_dataset, validation_data=val_tf_dataset, epochs=epochs)

# Evaluate the model
loss, accuracy = model.evaluate(test_tf_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/10
     24/Unknown 35s 614ms/step - accuracy: 0.1213 - loss: 1.2338

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


24/24 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.1219 - loss: 1.2330 - val_accuracy: 0.2000 - val_loss: 1.1472
Epoch 2/10


W0000 00:00:1721949941.069179     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 521ms/step - accuracy: 0.2068 - loss: 1.1463 - val_accuracy: 0.4400 - val_loss: 1.0744
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 521ms/step - accuracy: 0.4435 - loss: 1.0702 - val_accuracy: 0.4800 - val_loss: 1.0267
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 521ms/step - accuracy: 0.5112 - loss: 1.0278 - val_accuracy: 0.5100 - val_loss: 0.9955
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 20s 521ms/step - accuracy: 0.5246 - loss: 0.9928 - val_accuracy: 0.5500 - val_loss: 0.9761
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 521ms/step - accuracy: 0.5306 - loss: 0.9818 - val_accuracy: 0.5500 - val_loss: 0.9637
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 521ms/step - accuracy: 0.5588 - loss: 0.9506 - val_accuracy: 0.5500 - val_loss: 0.9553
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 522ms/step - accuracy: 0.5315 - loss: 0.9579 - val_accuracy: 0.5500 - val_loss: 0.9509
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 522ms/step - accuracy: 0.5692 - loss: 0.9340 - val_accuracy: 0.550

W0000 00:00:1721950075.699842     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5611 - loss: 0.9593
Test Loss: 0.9649049043655396
Test Accuracy: 0.5400000214576721


In [53]:
model.summary()

Model: "pub_med_qa_model_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_8 (Dropout)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │        24,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,123 (289.55 KB)

 Trainable params: 24,707 (96.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 49,416 (193.04 KB)

### 3.1.5. PubMedBert

## 3.2. Models 

### 3.2.1. Match-LSTM

### 3.2.2. BiDAF

# 4. Evaluate the results